# Uso de ficheros con DataFrames

## Carga de datos

#### Desde un fichero CSV

In [0]:
# Lectura del fichero indicando que tiene una fila de cabecera
dfCSV = spark.read.csv("/FileStore/tables/datosPaises.csv", header=True)
dfCSV.show(5)

+-------+----+------+------+
|   Pais|Edad|Sueldo|Compra|
+-------+----+------+------+
| France|  44| 72000|    No|
|  Spain|  27| 48000|   Yes|
|Germany|  30| 54000|    No|
|  Spain|  38| 61000|    No|
|Germany|  40| 57800|   Yes|
+-------+----+------+------+
only showing top 5 rows



In [0]:
# Método de escritura sobreescribiendo el archivo si existe
dfCSV.write.format("csv").mode("overwrite").save("/FileStore/tables/datosPaisesOut.csv")

Ejemplo de lectura de un fichero CSV que se guarda en formato JSON

In [0]:
df = spark.read.option("delimiter",";").option("header", "true").csv("/FileStore/tables/pdi_sales_small.csv")
df.printSchema()

root
 |-- ProductID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Revenue: string (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
df.write.json("/FileStore/tables/pdi_sales_json")

## Datos y esquemas

In [0]:
# Definición de un esquema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType, DoubleType

In [0]:
#StructType define un conjunto de columnas
#StructField define cada una de las columnas

esquema = StructType([StructField('Country', StringType(), True), 
                      StructField('CountryCode', StringType(), True), 
                      StructField('Year', IntegerType(), True), 
                      StructField('GDP', FloatType(), True)
                     ])

In [0]:
# Carga de los datos del DataFrame
df1 = spark.read.csv('/FileStore/tables/country_data.csv', header=True, schema=esquema)

In [0]:
# Muestra de los primeros seis registros
df1.show(6)

+----------+-----------+----+-------------+
|   Country|CountryCode|Year|          GDP|
+----------+-----------+----+-------------+
|Arab World|        ARB|1968| 2.5760684E10|
|Arab World|        ARB|1969|2.84342026E10|
|Arab World|        ARB|1970|3.13854996E10|
|Arab World|        ARB|1971| 3.6426912E10|
|Arab World|        ARB|1972| 4.3316056E10|
|Arab World|        ARB|1973| 5.5018394E10|
+----------+-----------+----+-------------+
only showing top 6 rows



In [0]:
df1.printSchema()

root
 |-- Country: string (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- GDP: float (nullable = true)



#### Otro ejemplo de generación de esquema

Se le asigna el esquema al DataFrame al leer los datos desde un fichero utilizando el método _schema_

In [0]:
# Se crea el esquema para el fichero datosPaises.csv usado al principio del notebook

esquema = StructType([
    StructField("Pais", StringType(), False),
    StructField("Edad", IntegerType(), False),
    StructField("Sueldo", DoubleType(), True),
    StructField("Compra", StringType(), False)
])

In [0]:
# Asignamos el esquema al leer el fichero
dfPaises = spark.read.option("header", True).schema(esquema).csv("/FileStore/tables/datosPaises.csv")

In [0]:
dfPaises.show(3)

+-------+----+-------+------+
|   Pais|Edad| Sueldo|Compra|
+-------+----+-------+------+
| France|  44|72000.0|    No|
|  Spain|  27|48000.0|   Yes|
|Germany|  30|54000.0|    No|
+-------+----+-------+------+
only showing top 3 rows



In [0]:
# Obtención de las columnas del DataFrame
dfPaises.columns

Out[17]: ['Pais', 'Edad', 'Sueldo', 'Compra']

In [0]:
# Tipos de datos de cada columna
dfPaises.dtypes

Out[18]: [('Pais', 'string'),
 ('Edad', 'int'),
 ('Sueldo', 'double'),
 ('Compra', 'string')]

In [0]:
# Esquema del DataFrame
dfPaises.schema

Out[19]: StructType(List(StructField(Pais,StringType,true),StructField(Edad,IntegerType,true),StructField(Sueldo,DoubleType,true),StructField(Compra,StringType,true)))

#### Cambio de tipo de columnas
Los tipos a utilizar deben estar importados previamente

In [0]:
# Cambio de tipo a Integer sobre la columna Sueldo
dfPaises2 = dfPaises.withColumn("Sueldo", dfPaises.Sueldo.cast(IntegerType()))

In [0]:
dfPaises2.printSchema()

root
 |-- Pais: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sueldo: integer (nullable = true)
 |-- Compra: string (nullable = true)



In [0]:
# Otra forma
dfPaises3 = dfPaises.withColumn("Sueldo", dfPaises.Sueldo.cast("integer"))

In [0]:
dfPaises3.schema

Out[23]: StructType(List(StructField(Pais,StringType,true),StructField(Edad,IntegerType,true),StructField(Sueldo,IntegerType,true),StructField(Compra,StringType,true)))